In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# 🧠 AGI Voice Agent - Memory Module (Notebook 15)

### Objective:
Implement a memory system that:
- Remembers past conversations
- Uses embeddings for semantic search
- Stores memory in Postgres + pgvector OR in-memory FAISS






📌 1. Install Dependencies

!pip install pgvector psycopg2-binary langchain sentence-transformers




📌 2. Setup Memory Database (Postgres + pgvector)

import psycopg2

conn = psycopg2.connect(
    dbname="agi_memory",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS memory (
    id SERIAL PRIMARY KEY,
    user_id TEXT,
    message TEXT,
    embedding VECTOR(384)
)
""")
conn.commit()
print("✅ Memory table ready.")




📌 3. Embedding Model

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')




📌 4. Store Message in Memory

def store_message(user_id, message):
    embedding = model.encode([message])[0]
    insert_query = """
    INSERT INTO memory (user_id, message, embedding)
    VALUES (%s, %s, %s)
    """
    cursor.execute(insert_query, (user_id, message, list(embedding)))
    conn.commit()
    print(f"Stored message for user {user_id}")

store_message("user1", "What is the weather today?")




📌 5. Retrieve Memory via Embedding Similarity

def retrieve_memory(user_id, new_message, top_k=3):
    new_embedding = model.encode([new_message])[0].tolist()
    
    search_query = """
    SELECT message, embedding <=> %s AS distance
    FROM memory
    WHERE user_id = %s
    ORDER BY distance ASC
    LIMIT %s
    """
    cursor.execute(search_query, (new_embedding, user_id, top_k))
    results = cursor.fetchall()
    return [msg for msg, _ in results]

memory = retrieve_memory("user1", "Tell me about today’s forecast.")
print("Memory Retrieved:", memory)




📌 6. Bonus: In-Memory FAISS Alternative

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
faiss_store = FAISS.from_texts(["sample conversation"], embedding_model)




📌 7. Next Steps

✅ Integrate memory retrieval before every model prompt

✅ Persist memory per user/session

✅ Proceed to:

18_Streamlit_UI_Development.ipynb for app frontend

Or FastAPI API layer

SyntaxError: invalid character '📌' (U+1F4CC) (4176838474.py, line 14)